In [1]:
import os;
import sys;
import random;
import json

import numpy as np;
import cv2;
import matplotlib.pyplot as plt;

import tensorflow as tf
from tensorflow import keras;
import openslide
import imutils

import pylab




from skimage.data import astronaut
from skimage.color import rgb2gray
from skimage.filters import sobel;
import skimage
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float


import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color

from skimage.filters.rank import entropy
from skimage.morphology import disk

from skimage.feature import local_binary_pattern


import segmentation_models as sm

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Segmentation Models: using `keras` framework.


Using TensorFlow backend.


In [2]:
datapath="./gland/";

train_ids=[];
test_ids=[];
test_ids2=[];

files=os.listdir(datapath)
for i in files:
    i = i.replace("_anno.bmp","").replace(".bmp","");
    if i.startswith("testB"):
        test_ids2.append(i)
        
    elif i.startswith("train"):
        train_ids.append(i)
        
    elif i.startswith("testA"):
        test_ids.append(i)
        

train_ids=list(set(train_ids))
test_ids=list(set(test_ids))
test_ids2=list(set(test_ids2))


print(len(train_ids))
print(len(test_ids))
print(len(test_ids2))

85
60
20


In [3]:
class DataGen(keras.utils.Sequence):
    def __init__(self,ids,batch_size=0,image_size=(256,256)):
        self.ids=ids;
        
        self.batch_size =batch_size;
        self.image_size = image_size;
        
        
    def __load__(self,id_name):
        
        path=datapath;
        
        image_path = os.path.join(path,id_name)+".bmp";
        
        image = cv2.imread(image_path);
        
        mask_path= os.path.join(path,id_name)+"_anno.bmp";
        mask = cv2.imread(mask_path,-1);
        
        thresh =0.5
        mask = cv2.threshold(mask, thresh, 255, cv2.THRESH_BINARY)[1]
        
        image =  cv2.normalize(image, None, 0, 255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC3)
        
        image = cv2.resize(image,self.image_size);
        
        
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY);
        
        
        deep=cv2.CV_64F;
        x=cv2.Sobel(gray,deep,1,0)
        y=cv2.Sobel(gray,deep,0,1)

        absx= cv2.convertScaleAbs(x)
        absy= cv2.convertScaleAbs(y)

        dist = cv2.addWeighted(absx,0.5,absy,0.5,0)
        
        lap = cv2.Laplacian(gray,deep,3)
        lap = cv2.convertScaleAbs(lap);
        
        result=np.zeros((self.image_size[0],self.image_size[1],5))
        result[:,:,0:3]=image;
        result[:,:,3]=dist
        result[:,:,4]=lap
        
        image= result;
        
        mask=cv2.resize(mask,self.image_size);
        mask = np.expand_dims(mask,axis=-1);
        mask =np.maximum(mask,mask);
        
        
        
        image =image/255.0;
        
        mask = mask/255.0;
        
        
        return image, mask;
    
    def __getitem__(self,index):
        if (index+1)*self.batch_size >len(self.ids):
            self.bach_size=len(self.ids)-index*batch_size;
            
        files_batch = self.ids[index*self.batch_size:(index+1)*self.batch_size]
        
        image=[];
        mask=[];
        
        for id_name in files_batch:
            _img,_mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask);
        
        image =np.array(image)
        mask =np.array(mask);
        
        return image, mask;
    
    def on_epoch_end(self):
        pass;
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)));

In [4]:
image_size = (256,256);
batch_size = 10

In [5]:
def down_block(x,filters,kernel_size=(3,3),padding="same",strides=1):
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(x);
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(c);
    p = keras.layers.MaxPool2D((2,2),(2,2))(c);
    
    return c,p

def up_block(x,skip,filters,kernel_size=(3,3),padding="same",strides=1):
    us=keras.layers.UpSampling2D((2,2))(x);
    concat = keras.layers.Concatenate()([us,skip])
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(concat);
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(c);
    return c;


def bottleneck(x,filters,kernel_size=(3,3),padding="same",strides=1):
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(x);
    c = keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation="relu")(c);
    return c;

In [6]:
def UNet():
    f = [16,32,64,128,256,512];
    
    inputs = keras.layers.Input(image_size+(5,))
    
    p0=inputs;
    c1,p1 = down_block(p0,f[0])  
    c2,p2 = down_block(p1,f[1]) 
    c3,p3 = down_block(p2,f[2])
    #c5,p5 = down_block(p4,f[4])
    
    #bn = bottleneck(p5,f[5])
    bn = bottleneck(p3,f[3])
    
    #u5= up_block(bn,c5,f[4])
    #u4= up_block(u5,c4,f[3])
    u3= up_block(bn,c3,f[2])
    u2= up_block(u3,c2,f[1])
    u1= up_block(u2,c1,f[0])
    
    outputs = keras.layers.Conv2D(1,(1,1),padding="same",activation='sigmoid')(u1);
    model = keras.models.Model(inputs,outputs)
    
    return model

In [7]:
smooth=1
def dice_coef(y_true,y_pred):
    
    y_true_f = keras.layers.Flatten()(y_true)
    y_pred_f = keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f* y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f)+tf.reduce_sum(y_pred_f)+ smooth )
    
def dice_coef_loss(y_true,y_pred):
    return 1.0- dice_coef(y_true,y_pred)

In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
model = UNet();

In [10]:
model.compile(optimizer = "adam",loss="binary_crossentropy",metrics=['acc',f1_m,precision_m, recall_m,dice_coef_loss]);


In [ ]:


train_data = DataGen(train_ids,batch_size=batch_size,image_size = image_size)
valid_data = DataGen(test_ids,batch_size=batch_size,image_size = image_size)
    
train_steps= len(train_ids)//batch_size;
valid_steps= len(test_ids)//batch_size;
    
epochs = 150;
#model.load_weights("model_test_check.hdf5")
result = model.fit_generator(train_data, validation_data=valid_data, 
                                 steps_per_epoch=train_steps, 
                                 validation_steps=valid_steps, 
                                 epochs=epochs,
                                )

Epoch 1/150
8/8 [==============================] - 22s 3s/step - loss: 0.6948 - acc: 0.5044 - f1_m: 0.4205 - precision_m: 0.4885 - recall_m: 0.5746 - dice_coef_loss: 0.4920 - val_loss: 0.6930 - val_acc: 0.4989 - val_f1_m: 1.4034e-04 - val_precision_m: 0.3815 - val_recall_m: 7.0181e-05 - val_dice_coef_loss: 0.5206
Epoch 2/150
8/8 [==============================] - 21s 3s/step - loss: 0.6905 - acc: 0.4955 - f1_m: 5.9686e-05 - precision_m: 0.5337 - recall_m: 2.9847e-05 - dice_coef_loss: 0.5232 - val_loss: 0.6937 - val_acc: 0.4990 - val_f1_m: 9.5856e-04 - val_precision_m: 0.6589 - val_recall_m: 4.7966e-04 - val_dice_coef_loss: 0.5206
Epoch 3/150
8/8 [==============================] - 21s 3s/step - loss: 0.6895 - acc: 0.5055 - f1_m: 0.1876 - precision_m: 0.7242 - recall_m: 0.1347 - dice_coef_loss: 0.5030 - val_loss: 0.6940 - val_acc: 0.4938 - val_f1_m: 0.3179 - val_precision_m: 0.4773 - val_recall_m: 0.2412 - val_dice_coef_loss: 0.5110
Epoch 4/150
8/8 [==============================] - 22s 

In [ ]:
history = str(result.history)

In [ ]:
model.save("unet_5c.hdf5")

In [ ]:
with open("unet_5c_history.json","w") as f:
    json.dump(history,f)